# Visualizing Maxar Open Data with Leafmap


The [Maxar Open Data Program](https://www.maxar.com/open-data) provides pre- and post-event high-resolution satellite imagery in support of emergency planning, risk assessment, monitoring of staging areas and emergency response, damage assessment, and recovery. Check out the links below for more information.
- [Maxar Open Data Program](https://www.maxar.com/open-data)
- [Maxar Open Data on AWS](https://registry.opendata.aws/maxar-open-data/)
- [Maxar Open Data on STAC Index](https://stacindex.org/catalogs/maxar-open-data-catalog-ard-format#/)
- [Maxar Open Data on STAC Browser](https://radiantearth.github.io/stac-browser/#/external/maxar-opendata.s3.amazonaws.com/events/catalog.json?.language=en)

The Maxar Open Data STAC catalog URL is: https://maxar-opendata.s3.amazonaws.com/events/catalog.json

Prepare Working Directory

In [ ]:
# Import the os module
import os

path = 'tmp/'

try:
    os.chdir(path)
    print("Current working directory: {0}".format(os.getcwd()))
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except NotADirectoryError:
    print("{0} is not a directory".format(path))
except PermissionError:
    print("You do not have permissions to change to {0}".format(path))

Import Libaray

In [ ]:
import leafmap

Retrieve all collections from the Maxar Open Data STAC catalog. Each collection represents a single event.

In [ ]:
leafmap.maxar_collections()

Retrieve all collections for a specific event:

In [ ]:
collections = leafmap.maxar_child_collections('Kahramanmaras-turkey-earthquake-23')
print(f"The number of collections: {len(collections)}")

In [ ]:
collections[:5]

Retrieve all items (tiles) for a specific collection and generate the footprints:

In [ ]:
gdf = leafmap.maxar_items(
    collection_id='Kahramanmaras-turkey-earthquake-23',
    child_id='1050050044DE7E00',
    return_gdf=True,
    assets=['visual'],
)
gdf.head()

Add the footprints to the map:

In [ ]:
m = leafmap.Map()
m.add_gdf(gdf, layer_name="Footprints")
m

Retrieve the COG URLs for all tiles in a collection:

In [ ]:
images = gdf['visual'].tolist()
images[:5]

Create a mosaic json file for the collection. You need to install `cogeo-mosaic` first using `pip install cogeo-mosaic`. Creating a mosaic json file might take a few minutes. Please be patient.

In [ ]:
leafmap.create_mosaicjson(images, output='1050050044DE7E00.json')

Make the mosaic json file available on the web, then you can add the mosaic to the map:

In [ ]:
m = leafmap.Map()
url = 'https://geospatial.glitch.me/1050050044DE7E00.json'
m.add_stac_layer(url, name="Mosaic")
m.add_gdf(gdf, layer_name="Footprints")
m

Retrieve the footprint of all tiles for a specific event. This might take 15+ minutes. Please be patient.

In [ ]:
# gdf = leafmap.maxar_all_items(
#     collection_id='Kahramanmaras-turkey-earthquake-23',
#     return_gdf=True,
#     verbose=True
#     )
# gdf.to_file('maxar_footprints.geojson', driver='GeoJSON')
# gdf

Add the footprints to the map:

In [ ]:
m = leafmap.Map(center=[36.844461, 37.386475], zoom=8)
# m.add_gdf(gdf, layer_name="Footprints")
url = 'https://cdn.glitch.global/cc5b7737-d8d0-4b07-bf2f-867b9009e986/maxar_footprints.geojson?v=1676583955698'
m.add_geojson(url, layer_name="Footprints")
m